In [1]:
import pandas as pd
import pickle
import matplotlib
import re
from textblob import TextBlob

%matplotlib inline

In [2]:
news = pd.read_pickle("data/scraped/newsFinal.pkl")
aapl = pd.read_pickle("data/stocks/aaplFinal.pkl")

In [3]:
print(news.info())
print(aapl.info())

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1693 entries, 2018-09-22 00:05:54 to 2018-11-06 15:18:29
Data columns (total 8 columns):
author         1646 non-null object
content        1541 non-null object
description    1693 non-null object
publishedAt    1693 non-null object
source         1693 non-null object
title          1693 non-null object
url            1693 non-null object
urlToImage     1566 non-null object
dtypes: object(8)
memory usage: 119.0+ KB
None
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 31 entries, 2018-09-25 to 2018-11-06
Data columns (total 7 columns):
open         31 non-null float64
high         31 non-null float64
low          31 non-null float64
close        31 non-null float64
volume       31 non-null int64
abschange    31 non-null float64
relchange    31 non-null float64
dtypes: float64(6), int64(1)
memory usage: 1.9 KB
None


In [4]:
aapl.drop(columns=["volume"], inplace=True)
news.drop(columns=['author','url','urlToImage','publishedAt','source','content'], inplace=True)

In [5]:
desc=[]
title=[]

In [6]:
for index, row in news.iterrows():
    desc.append(row['description'])
    title.append(row['title'])

In [7]:
REPLACE_NO_SPACE = re.compile("(\.)|(\;)|(\:)|(\!)|(\')|(\?)|(\,)|(\")|(\()|(\))|(\[)|(\])")
REPLACE_WITH_SPACE = re.compile("(<br\s*/><br\s*/>)|(\-)|(\/) \$")

In [8]:
def pre_process(content):
    
    content = [REPLACE_NO_SPACE.sub("", line.lower()) for line in content if line]
    content = [REPLACE_WITH_SPACE.sub(" ", line.lower()) for line in content if line]
    
    return content

def get_sentiment(content):
    sentiment = []
    for key, val in enumerate(content):
        wiki = TextBlob(val)
        sentiment.append(wiki.sentiment[0])
    return sentiment

In [9]:
desc = pre_process(desc)
title = pre_process(title)

In [10]:
news["title_score"] = get_sentiment(title)
news["desc_score"] = get_sentiment(desc)

In [11]:
news.drop(columns=["title","description"], inplace=True)
news = news.resample('B').mean()
news.head()

,title_score,desc_score
timestamp,,
2018-09-21,0.157076,0.131759
2018-09-24,0.104930,0.110943
2018-09-25,0.111867,0.161199
2018-09-26,0.091782,0.063646
2018-09-27,0.092848,0.207833


In [12]:
news.to_pickle("data/sentiment.pkl")

In [14]:
combined = pd.merge(news, aapl, how='inner', on="timestamp")
combined.to_pickle("data/final.pkl")